In [1]:
import random
import numpy as np
import pandas as pd
from transformers import *
from utils import get_word_embeddings, get_bert_vector, get_avg_similarity, get_bert_vector_dataset

In [18]:
df = pd.read_csv(r'data\MedicalConcepts_augmented.csv')
df.head()

,Term1,Term2,label
0,mittelschmerz,actonel,1
1,incontinence,lasix,1
2,hyperemesis,free,0
3,myositis,lipitor,1
4,plavix,ambivalent,0


In [19]:
related_terms = df[df.label==1]
related_terms.head()

,Term1,Term2,label
0,mittelschmerz,actonel,1
1,incontinence,lasix,1
3,myositis,lipitor,1
8,dysuria,cipro,1
9,enalapril,lisinopril,1


In [20]:
unrelated_terms = df[df.label==0]
unrelated_terms.head()

,Term1,Term2,label
2,hyperemesis,free,0
4,plavix,ambivalent,0
5,drooling,changes,0
6,otitis,clamping,0
7,clubbing,stilts,0


In [21]:
all_terms = sorted(set(df.Term1.tolist() + df.Term2.tolist()))

In [6]:
glove_embeddings = get_word_embeddings(
      r'C:\Users\Anna\Files\SemLex\embeddings\glove\glove-embeddings\glove.6B.100d.txt')

In [7]:
fasttext_embeddings = get_word_embeddings(
      r'C:\Users\Anna\Files\SemLex\embeddings\fasttext_aligned_cleaned\wiki.en.align.vec')

In [8]:
tokenizer_scibert = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
model_scibert = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/config.json from cache at C:\Users\Anna/.cache\huggingface\transformers\858852fd2471ce39075378592ddc87f5a6551e64c6825d1b92c8dab9318e0fc3.03ff9e9f998b9a9d40647a2148a202e3fb3d568dc0f170dda9dda194bab4d5dd
Model config BertConfig {
  "_name_or_path": "allenai/scibert_scivocab_uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.16.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31090
}

loading file htt

In [9]:
tokenizer_biobert = AutoTokenizer.from_pretrained('dmis-lab/biobert-v1.1')
model_biobert = AutoModel.from_pretrained('dmis-lab/biobert-v1.1')

loading configuration file https://huggingface.co/dmis-lab/biobert-v1.1/resolve/main/config.json from cache at C:\Users\Anna/.cache\huggingface\transformers\f048b8136bae2b3abe91e9e82949295fb205887c84db3be2775e1cdb0ecfeeb9.d7812d36d3371e4d43299a0c4a938622c5251db0efa17a5d4d9b57037fcec823
Model config BertConfig {
  "_name_or_path": "dmis-lab/biobert-v1.1",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.16.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file https://huggingface.co/dmis-lab/biobert-v

In [10]:
tokenizer_bert = BertTokenizer.from_pretrained('bert-base-uncased')
config = BertConfig()
model_bert = BertModel.from_pretrained('bert-base-uncased')
model_bert_mlm = BertModel.from_pretrained(r'models\bert_mlm')
model_bert_fine_tuned = BertModel.from_pretrained(r'models\bert_fine_tuned')

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at C:\Users\Anna/.cache\huggingface\transformers\45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at C:\Users\Anna/.cache\huggingface\transformers\c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json from cache at C:\Users\Anna/.cache\huggingface\transformers\534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca

Some weights of BertModel were not initialized from the model checkpoint at models\bert_fine_tuned and are newly initialized: ['encoder.layer.9.output.LayerNorm.weight', 'encoder.layer.3.attention.output.LayerNorm.weight', 'encoder.layer.4.attention.output.LayerNorm.weight', 'encoder.layer.11.attention.output.dense.bias', 'encoder.layer.6.attention.output.LayerNorm.bias', 'encoder.layer.1.attention.output.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.2.attention.output.LayerNorm.bias', 'encoder.layer.5.output.LayerNorm.weight', 'encoder.layer.9.attention.self.query.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.attention.self.key.weight', 'encoder.layer.7.intermediate.dense.bias', 'encoder.layer.5.attention.output.dense.bias', 'encoder.layer.10.output.LayerNorm.bias', 'encoder.layer.11.attention.output.dense.weight', 'pooler.dense.weight', 'embeddings.token

In [22]:
embeddings_scibert = get_bert_vector_dataset(all_terms, model_scibert, tokenizer_scibert)

In [23]:
embeddings_biobert = get_bert_vector_dataset(all_terms, model_biobert, tokenizer_biobert)

In [24]:
embeddings_bert = get_bert_vector_dataset(all_terms, model_bert, tokenizer_bert)

In [25]:
embeddings_bert_mlm = get_bert_vector_dataset(all_terms, model_bert_mlm, tokenizer_bert)

In [26]:
embeddings_bert_fine_tuned = get_bert_vector_dataset(all_terms, model_bert_fine_tuned, tokenizer_bert)

In [27]:
name_to_method = {'glove': (glove_embeddings, 'non_contextual'),
                 'fasttext': (fasttext_embeddings, 'non_contextual'),
                 'scibert': (embeddings_scibert, 'bert'),
                 'biobert': (embeddings_biobert, 'bert'),
                 'bert': (embeddings_bert, 'bert'),
                 'bert_mlm': (embeddings_bert_mlm, 'bert'),
                 'bert_fine_tuned': (embeddings_bert_fine_tuned, 'bert')}

In [28]:
for name, (embeddings, embedding_type) in name_to_method.items():
    print(name)
    print('Number of missing terms:', len([term for term in all_terms if term not in embeddings.keys()]))
    avg_sim_related = get_avg_similarity(related_terms.Term1.tolist(), related_terms.Term2.tolist(), embeddings, embedding_type)
    avg_sim_unrelated = get_avg_similarity(unrelated_terms.Term1.tolist(), unrelated_terms.Term2.tolist(), embeddings, embedding_type)
    avg_sim_unrelated_non_med = get_avg_similarity(unrelated_terms.Term1.tolist(), unrelated_terms.Term2.tolist(), embeddings, embedding_type)
    print('avg_sim_related:', avg_sim_related)
    print('avg_sim_unrelated:', avg_sim_unrelated)

glove
Number of missing terms: 283
avg_sim_related: [[0.30563128]]
avg_sim_unrelated: [[0.05330584]]
fasttext
Number of missing terms: 104
avg_sim_related: [[0.47708768]]
avg_sim_unrelated: [[0.26278204]]
scibert
Number of missing terms: 0
avg_sim_related: [[0.8408807]]
avg_sim_unrelated: [[0.7975625]]
biobert
Number of missing terms: 0
avg_sim_related: [[0.95431215]]
avg_sim_unrelated: [[0.93287104]]
bert
Number of missing terms: 0
avg_sim_related: [[0.88583654]]
avg_sim_unrelated: [[0.8622656]]
bert_mlm
Number of missing terms: 0
avg_sim_related: [[0.97700816]]
avg_sim_unrelated: [[0.971721]]
bert_fine_tuned
Number of missing terms: 0
avg_sim_related: [[0.93542624]]
avg_sim_unrelated: [[0.92965233]]
